# Exercise 3: Markovian Drone

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#### Exercise 3.1
Implement the remaining functions of `DroneMDP` to compute the optimal policy via value iteration. Specifically, implement the functions:
1. `compute_action_values`: computes the action-values, $Q(x,u) \coloneqq R(x, u) + \gamma \sum_{x'} p(x' \:|\: x, u) V(x')$ for all $x \in \mathcal{X}$
2. `value_iteration`: performs value iteration to compute the optimal value function
3. `optimal_policy`: extracts the optimal policy from the optimal value function

In [ ]:
class DroneMDP(object):
    def __init__(self, n=20, γ=0.95, σ=10, x_eye=np.array([15, 15]), x_goal=np.array([19, 9])):
        self.n = n # grid size
        m = 4 # number of control inputs
        self.γ = γ
        self.x_goal = x_goal

        # Storm params
        self.x_eye = x_eye
        self.σ = σ

        # Control input set: 0 = `up`, 1 = `down`, 2 = `left`, 3 = `right`
        self.U = np.arange(m)

        # Enumerate the transition probabilities for each state and control, from
        # each possible current state
        # NOTE: `p(x_next; x, u) = P[x[0], x[1], u, x_next[0], x_next[1]]`
        self.P = np.zeros((n, n, m, n, n))
        for i in range(n):
            for j in range(n):
                # Start by identifying the set of all possible next states
                X_next = np.clip(np.array([
                    [i, j + 1],              # up
                    [i, j - 1],              # down
                    [i - 1, j],              # left
                    [i + 1, j],              # right
                ]), 0, n - 1)                # clip to grid
        
                # If the drone is "caught" by the storm, it may move in any given
                # direction with probability `w*0.25`
                x = np.array([i, j])
                w = np.exp(-np.sum((x - x_eye)**2) / (2*(σ**2)))
                for u in self.U:
                    np.add.at(self.P, (i, j, u, X_next[:, 0], X_next[:, 1]), w*0.25)
        
                # Otherwise, the drone moves in the chosen direction with
                # probability `1 - w`
                self.P[i, j, self.U, X_next[:, 0], X_next[:, 1]] += 1 - w          

        # Define reward matrix, R(x,u) = R[x[0], x[1], u]
        self.R = -np.ones((n, n, m))
        self.R[x_goal[0], x_goal[1], :m] = 0.

    def simulate(self, x_start, π, N=100):
        """
        Simulate the drone MDP given a starting position and a policy.

        Parameters:
            x_start: starting position
            π: policy, a np.ndarray of shape (n, n)
            N: number of steps to simulate

        Returns:
            np.ndarray of shape (N, 2) for the trajectory
        """
        rng = np.random.default_rng(0)  # seed RNG for reproducibility
        x = np.zeros((N, 2)).astype(int)
        x[0] = x_start
        dx = np.array([[0, 1], [0, -1], [-1, 0], [1, 0]])
        for k in range(N - 1):
            u = π[x[k, 0], x[k, 1]]
            w = np.exp(-np.sum((x[k] - self.x_eye)**2) / (2*(self.σ**2)))
            P_next = self.P[x[k, 0], x[k, 1], u, :, :]
            flat_index = rng.choice(P_next.size, p=P_next.ravel())
            x[k+1] = np.unravel_index(flat_index, P_next.shape)
        return x

    def compute_action_values(self, V):
        """
        Compute array of action-values Q(x,u) = R(x,u) + γ sum_x' p(x'; x,u)V(x')
        for all x in the statespace.

        Parameters:
            V: np.ndarray of shape (n, n) of the value function for each state in the grid

        Returns:
            np.ndarray of shape (n, n, m) for the action-value function, Q
        """
        ##### YOUR CODE STARTS HERE #####
        # Hint: Use self.R, self.γ, and self.P

        ###### YOUR CODE END HERE ######

    def value_iteration(self, eps=1e-4, max_iters=1000):
        """
        Use value iteration to compute the optimal value function, V. We 
        consider the algorithm to have converged once all values are within
        `eps` of the previous iteration value.

        Parameters:
            eps: tolerance for convergence
            max_iters: max number of iterations to run

        Returns:
            np.ndarray of shape (n, n) for the optimal value function, V
        """
        converged = False
        V = np.zeros((self.n, self.n))
        ##### YOUR CODE STARTS HERE #####
        # Note: you need to set `converged` to True if successful

        ###### YOUR CODE END HERE ######
        if not converged:
            raise RuntimeError('Value iteration did not converge!')
        return V

    def optimal_policy(self, V_star):
        """
        Compute the optimal policy given the optimal value function.
        """
        ##### YOUR CODE STARTS HERE #####

        ###### YOUR CODE END HERE ######       

#### Exercise 3.2: Simulate and Plot
Run the code below to compute the optimal policy, simulate it, and plot the results.

In [ ]:
# Compute the optimal value function and policy via value iteration
mdp = DroneMDP()
V_star = mdp.value_iteration()
π_star = mdp.optimal_policy(V_star)

# Simulate the drone from a starting position
x_start=np.array([0, 19])
x = mdp.simulate(x_start, π_star, N=100)

# Plot the optimal value function as a heat map
fig, ax = plt.subplots(1, 1, dpi=150)
im = ax.imshow(V_star.T, origin='lower')
plt.colorbar(im, label=r'$V^*\!(x)$')
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.show()

# Plot the optimal policy as a heat map, and overlay the simulated trajectory
fig, ax = plt.subplots(1, 1, dpi=150)
im = ax.imshow(π_star.T, origin='lower')
plt.colorbar(im, label=r'$\pi^*\!(x)$')
ax.plot(mdp.x_goal[0], mdp.x_goal[1], '*', color='k', markersize=10)
ax.plot(mdp.x_eye[0], mdp.x_eye[1], '^', color='tab:orange', markersize=10)
ax.plot(x_start[0], x_start[1], 'o', color='tab:green', markersize=10)
ax.plot(x[:, 0], x[:, 1], color='tab:red')
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.show() 